In [1]:
import os
import zipfile
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
dataset_zip = 'enfermedades_piel.v1i.tensorflow.zip'
extract_dir = 'data/'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [3]:
train_csv_path = os.path.join(extract_dir, 'train', '_annotations.csv')
valid_csv_path = os.path.join(extract_dir, 'valid', '_annotations.csv')
test_csv_path = os.path.join(extract_dir, 'test', '_annotations.csv')

In [4]:
def load_data(csv_path, base_dir):
    data = pd.read_csv(csv_path)
    images = []
    labels = []
    for index, row in data.iterrows():
        img_path = os.path.join(base_dir, row['filename'])
        image = Image.open(img_path)
        image = image.resize((img_width, img_height))
        image = tf.keras.preprocessing.image.img_to_array(image)
        images.append(image)
        labels.append(row['class'])
    return np.array(images), np.array(labels)

In [5]:
batch_size = 32
img_height = 150
img_width = 150
epochs = 50

In [6]:
train_images, train_labels = load_data(train_csv_path, os.path.join(extract_dir, 'train'))
valid_images, valid_labels = load_data(valid_csv_path, os.path.join(extract_dir, 'valid'))
test_images, test_labels = load_data(test_csv_path, os.path.join(extract_dir, 'test'))

In [7]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)
test_labels_encoded = label_encoder.transform(test_labels)

In [8]:
num_classes = len(label_encoder.classes_)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes)
valid_labels_categorical = to_categorical(valid_labels_encoded, num_classes)
test_labels_categorical = to_categorical(test_labels_encoded, num_classes)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels_categorical, batch_size=batch_size)
validation_generator = validation_datagen.flow(valid_images, valid_labels_categorical, batch_size=batch_size)


### Train model

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27244\1813032623.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True)


In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(valid_images) // batch_size,
    callbacks=[early_stopping_callback]
)

Epoch 1/50


c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


110/110 ━━━━━━━━━━━━━━━━━━━━ 72s 610ms/step - accuracy: 0.6637 - loss: 0.9428 - val_accuracy: 0.8500 - val_loss: 0.4209
Epoch 2/50
  1/110 ━━━━━━━━━━━━━━━━━━━━ 46s 427ms/step - accuracy: 0.7500 - loss: 0.4941

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7500 - loss: 0.4941 - val_accuracy: 0.9333 - val_loss: 0.2261
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 45s 389ms/step - accuracy: 0.8564 - loss: 0.3652 - val_accuracy: 0.8562 - val_loss: 0.3735
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8438 - loss: 0.3947 - val_accuracy: 0.8000 - val_loss: 0.3367
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 45s 403ms/step - accuracy: 0.8627 - loss: 0.3380 - val_accuracy: 0.8594 - val_loss: 0.3483
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8438 - loss: 0.3384 - val_accuracy: 0.8000 - val_loss: 0.4233
Epoch 7/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 53s 475ms/step - accuracy: 0.8604 - loss: 0.3398 - val_accuracy: 0.8594 - val_loss: 0.3765
Epoch 8/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8125 - loss: 0.3665 - val_accuracy: 0.9333 - val_loss: 0.1847
Epoch 9/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 49s 429ms/step - accuracy: 0.8855 - loss: 0.2950 - val_accuracy: 0

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_images, test_labels_categorical, batch_size=batch_size)
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_images) // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 345ms/step - accuracy: 0.7847 - loss: 0.7394
Test accuracy: 81.25%


In [16]:
model.save('skin_disease_classifier2.h5')

## 2 (pretrain tuning)

In [19]:
batch_size = 32
img_height = 150
img_width = 150
epochs = 100

In [21]:
base_model2 = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model2.layers[-20:]:
    layer.trainable = True

model2 = tf.keras.Sequential([
    base_model2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27244\399603819.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model2 = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


In [22]:
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping and learning rate scheduler callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


In [23]:
history = model2.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(valid_images) // batch_size,
    callbacks=[early_stopping_callback, lr_scheduler_callback]
)

Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 238s 2s/step - accuracy: 0.6116 - loss: 0.9523 - val_accuracy: 0.6594 - val_loss: 0.8897 - learning_rate: 1.0000e-04
Epoch 2/100
  1/110 ━━━━━━━━━━━━━━━━━━━━ 2:59 2s/step - accuracy: 0.8438 - loss: 0.4846

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8438 - loss: 0.4846 - val_accuracy: 0.6333 - val_loss: 1.3181 - learning_rate: 1.0000e-04
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.8985 - loss: 0.2699 - val_accuracy: 0.7281 - val_loss: 1.1443 - learning_rate: 1.0000e-04
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 1.0000 - loss: 0.0629 - val_accuracy: 0.7000 - val_loss: 1.3160 - learning_rate: 1.0000e-04
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9275 - loss: 0.1901 - val_accuracy: 0.7250 - val_loss: 1.1685 - learning_rate: 1.0000e-04
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9375 - loss: 0.2354 - val_accuracy: 0.7667 - val_loss: 0.6926 - learning_rate: 1.0000e-04
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.9305 - loss: 0.1670 - val_accuracy: 0.7563 - val_loss: 1.0008 - learning_rate: 1.0000e-04
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 

In [24]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_images, test_labels_categorical, batch_size=batch_size)
test_loss, test_accuracy = model2.evaluate(test_generator, steps=len(test_images) // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 308ms/step - accuracy: 0.7655 - loss: 0.7812
Test accuracy: 75.63%


In [25]:
model2.save('skin_disease_classifier3.h5')

## 3 (Overfitting problem)

In [26]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,  # Increased rotation range
    width_shift_range=0.3,  # Increased width shift range
    height_shift_range=0.3,  # Increased height shift range
    shear_range=0.3,  # Increased shear range
    zoom_range=0.3,  # Increased zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Added vertical flip
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels_categorical, batch_size=batch_size)
validation_generator = validation_datagen.flow(valid_images, valid_labels_categorical, batch_size=batch_size)


In [27]:
base_model3 = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model3.trainable = True  # Unfreeze the base model

# Fine-tune from this layer onwards
fine_tune_at = 100

for layer in base_model3.layers[:fine_tune_at]:
    layer.trainable = False

model3 = tf.keras.Sequential([
    base_model3,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.6),  # Increased dropout rate
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Added extra dense layer for regularization
    tf.keras.layers.Dropout(0.6),  # Added extra dropout layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27244\2948030516.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model3 = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


In [28]:
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping and learning rate scheduler callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

history = model3.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(valid_images) // batch_size,
    callbacks=[early_stopping_callback, lr_scheduler_callback]
)

Epoch 1/100


c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


110/110 ━━━━━━━━━━━━━━━━━━━━ 94s 716ms/step - accuracy: 0.3952 - loss: 12.0615 - val_accuracy: 0.5188 - val_loss: 11.1763 - learning_rate: 1.0000e-04
Epoch 2/100
  1/110 ━━━━━━━━━━━━━━━━━━━━ 56s 519ms/step - accuracy: 0.7188 - loss: 10.7718

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7188 - loss: 10.7718 - val_accuracy: 0.5667 - val_loss: 11.1784 - learning_rate: 1.0000e-04
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 61s 546ms/step - accuracy: 0.7343 - loss: 10.5192 - val_accuracy: 0.5562 - val_loss: 10.5400 - learning_rate: 1.0000e-04
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8125 - loss: 9.7107 - val_accuracy: 0.6667 - val_loss: 10.3137 - learning_rate: 1.0000e-04
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 63s 566ms/step - accuracy: 0.8199 - loss: 9.5128 - val_accuracy: 0.5813 - val_loss: 9.9390 - learning_rate: 1.0000e-04
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8750 - loss: 8.7871 - val_accuracy: 0.5333 - val_loss: 9.5985 - learning_rate: 1.0000e-04
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 60s 535ms/step - accuracy: 0.8618 - loss: 8.6346 - val_accuracy: 0.6031 - val_loss: 9.1194 - learning_rate: 1.0000e-04
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - a

In [29]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(test_images, test_labels_categorical, batch_size=batch_size)
test_loss, test_accuracy = model3.evaluate(test_generator, steps=len(test_images) // batch_size)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

c:\Users\Lenovo\Pemrograman\Bangkit\cap-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step - accuracy: 0.9161 - loss: 0.7130
Test accuracy: 91.87%


In [30]:
model3.save('skin_disease_classifier4.h5')

## Predict

In [34]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the model
model3 = tf.keras.models.load_model('skin_disease_classifier4.h5')

# Define the image size
img_height = 150
img_width = 150

# Define the class labels (make sure these match your training classes)
class_labels = ["class1", "class2", "class3", ..., "classN"]  # Replace with your actual class labels

def load_and_preprocess_image(img_path, target_size):
    # Load the image
    img = image.load_img(img_path, target_size=target_size)
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    # Expand dimensions to match the shape the model expects (batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize the image (assuming the model expects values in range [0, 1])
    img_array /= 255.0
    
    return img_array

def predict_image(model, img_path, target_size):
    # Preprocess the image
    preprocessed_image = load_and_preprocess_image(img_path, target_size)
    
    # Make prediction
    prediction = model.predict(preprocessed_image)
    
    # Decode the prediction
    predicted_class = np.argmax(prediction, axis=-1)
    return class_labels[predicted_class[0]]

# Define the path to your image
image_path = 'data/test/normal-skin.jpg'  # Replace with the actual path to your image

# Predict the class of the image
predicted_class = predict_image(model3, image_path, (img_height, img_width))

# Print the prediction
print("Predicted class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: class3


In [35]:
print(num_classes)

4


In [15]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model_path = 'skin_disease_classifier4.h5'  # Change this to the path of your saved model
model = load_model(model_path)

image_path = 'data/test/Acne-copy.jpg'  # Change this to the path of your image

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = img / 255.0  # Normalize the pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

image = preprocess_image(image_path)

predictions = model.predict(image)
predicted_class = np.argmax(predictions)

predicted_disease_type = label_encoder.inverse_transform([predicted_class])[0]

print("Predicted Disease Type:", predicted_disease_type)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Disease Type: acne
